In [4]:
# Chapter 5 Gedanken V2.0
# Type: Subcritical surrogate dataset generator; Specifications: PHWR-style cylinder + Th-U233 fuel; ENDF/B-VIII.0 σ; 36^3 reference
# Outputs: CSV, Parquet, NPZ, dataset_schema.json, material_library.json, nuclear_data.json, EXFOR raw JSON data.

!pip -q install requests requests_cache pandas pyarrow scipy

import os, json, math, time, random
from dataclasses import dataclass, asdict, field
from typing import Optional, Tuple, Dict, Any

import numpy as np
import pandas as pd
import requests
import requests_cache
from scipy.special import j0

# Block 1 - Setting up Reproduction + Output paths
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

OUT_DIR = "step5_dataset_out"
EXFOR_RAW_DIR = os.path.join(OUT_DIR, "exfor_raw")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(EXFOR_RAW_DIR, exist_ok=True)

# Block 1.1 - Caching the EXFOR calls (Since we are using ENDF/B-VIII.0, Block 1.1 is not needed as of now)
requests_cache.install_cache("exfor_cache", backend="sqlite", expire_after=7 * 24 * 3600)

# Block 2 - Units & constants (For consistency)
# Length: cm
# Cross - section: barns
# Number density: atoms/cm^3
# Macro XS: 1/cm
# Flux: n/(cm^2·s)
# Energy: Joules

BARN_TO_CM2 = 1e-24          # Established data
E_FISSION_J = 3.204e-11      # ~200 MeV in Joules
NU_U233 = 2.50               # average ν for U-233 (thermal)
THERMAL_E_eV = 0.0253

# Calculating Pa-233's half life in days
PA233_HALF_LIFE_D = 26.975
PA233_LAMBDA_S = math.log(2.0) / (PA233_HALF_LIFE_D * 86400.0)

# Referencing heavy metal atomic number density (orders of magnitude realistic)
# UO2/ThO2 solids are approximately (2-6)e22 atoms/cm^3 heavy-metal; Choosing the mid range:
N_HM = 4.8e22  # (Atoms/cm^3 for heavy metal atoms only)

# Block 3 - Safety envelopes (Not used for optimization; Used for labelling/gating purposes)
KEFF_TARGET_MAX = 0.98
POWER_ENV_MAX_W = 5.0e8              # 500 MW thermal
POWER_DENSITY_MAX_W_PER_CM3 = 100.0

# Block 4 - Dataclasses
@dataclass
class NuclearData:
    sigma_f_U233_b: float
    sigma_c_U233_b: float
    sigma_ng_Th232_b: float
    nubar_U233: float
    lambda_Pa233_s: float
    used_exfor: bool
    source_info: Dict[str, Any] = field(default_factory=dict)

@dataclass
class ReactorConfig:
    core_radius_cm: float
    blanket_thickness_cm: float
    height_cm: float
    initial_U233_frac: float  # heavy-metal atomic fraction in fuel region

@dataclass
class SampleRecord:
    core_radius_cm: float
    blanket_thickness_cm: float
    height_cm: float
    initial_U233_frac: float

    core_aspect: float
    blanket_to_core_radius_ratio: float

    keff_est: float
    eta_est: float
    flux_shape_metric: float

    breeding_proxy_atoms: float
    breeding_proxy_norm: float

    total_power_W: float
    avg_core_power_density_W_cm3: float

    violates_power_envelope: bool
    violates_temp_envelope: bool

    margin_keff: float
    margin_power_W: float
    margin_temp_W_cm3: float
    safety_margin: float

# Block 5 - EXFOR API helpers (This Block is not contributing to the the Output since we are using data from ENDF)
EXFOR_BASES = [
    "https://www-nds.iaea.org/exfor/program",                    # IAEA (sometimes 403 from Colab)
    "https://www-nds.iaea.org/exfor/servlet/X4Interfaces",       # IAEA alternate endpoint
    "https://www.nndc.bnl.gov/exfor/servlet/X4Interfaces",       # NNDC mirror (often more permissive)
]


def _safe_filename(s: str) -> str:
    return "".join(c if (c.isalnum() or c in "-_.") else "_" for c in s)

EXFOR_HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; academic-research; +https://example.edu)",
    "Accept": "application/json,text/plain,*/*",
}

def _req_json(url: str, params: Dict[str, str], tag: str, max_tries: int = 4) -> Optional[Dict[str, Any]]:
    last_err = None

    for base in EXFOR_BASES:
        for attempt in range(1, max_tries + 1):
            try:
                r = requests.get(base, params=params, headers=EXFOR_HEADERS, timeout=20)
                if r.status_code == 200:
                    raw_path = os.path.join(EXFOR_RAW_DIR, f"{_safe_filename(tag)}_http200.json")
                    try:
                        with open(raw_path, "w", encoding="utf-8") as f:
                            f.write(r.text)
                    except Exception:
                        pass
                    return r.json()

                last_err = f"HTTP {r.status_code} @ {base}"
                # If forbidden, don’t hammer—move to next base quickly
                if r.status_code in (401, 403):
                    break

            except Exception as e:
                last_err = f"{str(e)} @ {base}"

            time.sleep(0.7 * attempt)

    fail_path = os.path.join(EXFOR_RAW_DIR, f"{_safe_filename(tag)}_FAILED.txt")
    with open(fail_path, "w", encoding="utf-8") as f:
        f.write(f"Failed after bases+tries. Last error: {last_err}\n")
        f.write(f"PARAMS: {params}\n")
    return None


def fetch_exfor_sigma_thermal(
    target: str,
    reaction: str,
    quantity: str = "SIG",
    e_target_eV: float = THERMAL_E_eV,
) -> Tuple[Optional[float], Dict[str, Any]]:
    """
    Returns (sigma_barns_at_nearest_energy, meta).
    Meta includes: source, entry, points, nearest_energy_eV.
    If not found, returns (None, meta).
    """
    # Initial - Normalize reaction: "(N,G)" -> "N,G" ; strip spaces; uppercase
    reaction = reaction.strip().upper()
    if reaction.startswith("(") and reaction.endswith(")"):
        reaction = reaction[1:-1].strip()
    reaction = reaction.replace(" ", "")

    # 1) list entries
    tag_list = f"x4list_{target}_{reaction}_{quantity}"
    params_list = {
        "x4list": "",             # flag parameter
        "Target": target,
        "Reaction": reaction,
        "Quantity": quantity,
        "out": "json",
    }
    j_list = _req_json(EXFOR_BASE, params_list, tag_list)
    if not j_list:
        return None, {"source": "EXFOR_error_list"}

    # EXFOR responses vary; try common keys
    results = None
    for k in ("result", "Result", "results", "Results"):
        if k in j_list and isinstance(j_list[k], list):
            results = j_list[k]
            break
    if not results:
        return None, {"source": "EXFOR_not_found"}

    # try to extract first entry id
    entry_id = None
    first = results[0]
    if isinstance(first, dict):
        for key in ("ENTRY", "Entry", "EntryID", "ENTRY_ID", "entry"):
            if key in first:
                entry_id = str(first[key])
                break
    if entry_id is None:
        # sometimes it's a string already
        entry_id = str(first)

    # 2) fetch entry data
    tag_get = f"x4get_{entry_id}_{target}_{reaction}_{quantity}"
    params_get = {
        "x4get": "",
        'Entry': entry_id,
        "out": "json",
    }
    j_get = _req_json(EXFOR_BASE, params_get, tag_get)
    if not j_get:
        # try alternate op
        params_get2 = {
            "x4get": "",
            "Entry": entry_id,
            "op": "x4jsfy",
        }
        j_get = _req_json(EXFOR_BASE, params_get2, tag_get + "_x4jsfy")
        if not j_get:
            return None, {"source": "EXFOR_error_get", "entry": entry_id}

    # try to locate numeric datasets; common patterns: "data", "DATA"
    data = None
    for k in ("data", "DATA"):
        if k in j_get and isinstance(j_get[k], list):
            data = j_get[k]
            break

    # sometimes nested: {"datasets":[{"data":[...]}]}
    if data is None and isinstance(j_get, dict):
        for k in ("datasets", "Datasets", "set", "SET"):
            if k in j_get and isinstance(j_get[k], list) and j_get[k]:
                d0 = j_get[k][0]
                if isinstance(d0, dict):
                    for kk in ("data", "DATA"):
                        if kk in d0 and isinstance(d0[kk], list):
                            data = d0[kk]
                            break
                if data is not None:
                    break

    if not data:
        return None, {"source": "EXFOR_no_data", "entry": entry_id}

    # filter points shaped like [E, value, ...]
    pts = []
    for row in data:
        if isinstance(row, (list, tuple)) and len(row) >= 2:
            try:
                E = float(row[0])
                v = float(row[1])
                if E > 0 and np.isfinite(v):
                    pts.append((E, v))
            except Exception:
                continue
    if not pts:
        return None, {"source": "EXFOR_no_points", "entry": entry_id}

    # choose nearest to target energy on log scale
    log_target = math.log10(e_target_eV)
    best = min(pts, key=lambda t: abs(math.log10(t[0]) - log_target))
    sigma_b = float(best[1])
    meta = {
        "source": "EXFOR",
        "entry": entry_id,
        "points": len(pts),
        "nearest_energy_eV": float(best[0]),
        "target": target,
        "reaction": reaction,
        "quantity": quantity,
    }
    # also write a compact extracted record
    compact_path = os.path.join(EXFOR_RAW_DIR, f"{_safe_filename(tag_get)}_EXTRACT.json")
    with open(compact_path, "w", encoding="utf-8") as f:
        json.dump({"meta": meta, "sample_points_head": pts[:10]}, f, indent=2)
    return sigma_b, meta

# Block 6 - Building Nuclear Dataset with ENDF anchoring and Safe defaults
used_exfor = False
source_info = {}

sigma_f_U233 = 534.072
sigma_c_U233 = 42.329
sigma_ng_Th232 = 7.33806

source_info["U233_fission"]  = {"source": "ENDF", "library": "ENDF/B-VIII.0", "E_eV": THERMAL_E_eV, "note": "manual thermal point 2.53e-8 MeV"}
source_info["U233_capture"]  = {"source": "ENDF", "library": "ENDF/B-VIII.0", "E_eV": THERMAL_E_eV, "note": "manual thermal point 2.53e-8 MeV"}
source_info["Th232_capture"] = {"source": "ENDF", "library": "ENDF/B-VIII.0", "E_eV": THERMAL_E_eV, "note": "manual thermal point 2.53e-8 MeV"}

nuc = NuclearData(
    sigma_f_U233_b=sigma_f_U233,
    sigma_c_U233_b=sigma_c_U233,
    sigma_ng_Th232_b=sigma_ng_Th232,
    nubar_U233=NU_U233,
    lambda_Pa233_s=PA233_LAMBDA_S,
    used_exfor=used_exfor,
    source_info=source_info,
)

with open(os.path.join(OUT_DIR, "nuclear_data.json"), "w", encoding="utf-8") as f:
    json.dump(asdict(nuc), f, indent=2)

# Block 7 - Defining Materials library (Documentary)
material_library = {
    "fuel_heavy_metal": {
        "description": "Fuel heavy metal mixture in core region: Th-232 + U-233 (fraction varies by sample).",
        "number_density_atoms_cm3": N_HM,
        "composition_template": {"Th232": "1 - initial_U233_frac", "U233": "initial_U233_frac"},
    },
    "blanket_heavy_metal": {
        "description": "Blanket heavy metal assumed Th-232 (for breeding proxy).",
        "number_density_atoms_cm3": N_HM,
        "composition": {"Th232": 1.0},
    },
    "moderator": {"description": "PHWR-style moderator (heavy water). Not explicitly modeled in one-group proxy.", "material": "D2O"},
    "reflector": {"description": "Reflector/structure boundary. Leakage approximated via buckling.", "material": "Graphite/structure"},
}
with open(os.path.join(OUT_DIR, "material_library.json"), "w", encoding="utf-8") as f:
    json.dump(material_library, f, indent=2)

# Block 8 - Physics helpers
def cylinder_vol_cm3(radius_cm: float, height_cm: float) -> float:
    return math.pi * radius_cm * radius_cm * height_cm

def macro_sigma(N_atoms_cm3: float, sigma_b: float) -> float:
    # Σ [1/cm] = N [atoms/cm3] * σ [barn] * 1e-24 [cm2/barn]
    return N_atoms_cm3 * sigma_b * BARN_TO_CM2

def keff_proxy(cfg: ReactorConfig) -> Tuple[float, float, float, float, float]:
    """
    Returns:
      keff_est, eta_est, Sigma_a_eff, D_clamped, B2
    One-group diffusion proxy:
      k = nu Σf / (Σa + D*B2)
    Σa, Σf are volume-weighted across core + blanket
    """
    Rc = cfg.core_radius_cm
    Rt = cfg.core_radius_cm + cfg.blanket_thickness_cm
    H  = cfg.height_cm
    if Rt <= 0 or H <= 0:
        return 0.0, 0.0, 0.0, 1.0, 0.0

    Vc = cylinder_vol_cm3(Rc, H)
    Vt = cylinder_vol_cm3(Rt, H)
    Vb = max(Vt - Vc, 0.0)
    Vtot = Vc + Vb

    # number densities in core
    fU = cfg.initial_U233_frac
    fU = min(max(fU, 0.0), 1.0)
    N_u = N_HM * fU
    N_th_core = N_HM * (1.0 - fU)
    N_th_blanket = N_HM

    # macroscopic in core
    Sig_f_core = macro_sigma(N_u, nuc.sigma_f_U233_b)
    Sig_cU_core = macro_sigma(N_u, nuc.sigma_c_U233_b)
    Sig_cTh_core = macro_sigma(N_th_core, nuc.sigma_ng_Th232_b)
    Sig_a_core = Sig_f_core + Sig_cU_core + Sig_cTh_core

    # blanket absorption only (no fissile)
    Sig_a_blanket = macro_sigma(N_th_blanket, nuc.sigma_ng_Th232_b)

    # volume-weighted Σa
    Sig_a_eff = (Sig_a_core * Vc + Sig_a_blanket * Vb) / Vtot

    # volume-weighted νΣf (only core contributes)
    nuSig_f_eff = nuc.nubar_U233 * Sig_f_core * (Vc / Vtot)

    # diffusion coefficient proxy D = 1/(3 Σa) clamped
    if Sig_a_eff <= 0:
        D = 1e6
    else:
        D = 1.0 / (3.0 * Sig_a_eff)
    D = min(max(D, 1e-6), 1e2)

    # geometric buckling (cylinder fundamental)
    B2 = (2.405 / Rt) ** 2 + (math.pi / H) ** 2

    denom = Sig_a_eff + D * B2
    keff = (nuSig_f_eff / denom) if denom > 0 else 0.0

    # eta proxy (infinite medium, no leakage)
    eta = (nuSig_f_eff / Sig_a_eff) if Sig_a_eff > 0 else 0.0
    return keff, eta, Sig_a_eff, D, B2

def flux_shape_metric(cfg: ReactorConfig) -> float:
    """
    Fundamental-mode radial flux ~ J0(2.405 r/Rt). Metric ~ 1 - J0(2.405 Rc/Rt).
    Clamped to [0,1]. Uses SciPy's j0 for correctness.
    """
    Rt = cfg.core_radius_cm + cfg.blanket_thickness_cm
    if Rt <= 0:
        return 0.0
    x = 2.405 * (cfg.core_radius_cm / Rt)
    edge = float(j0(x))
    edge = max(edge, 0.0)  # don't allow negative for this metric
    m = 1.0 - edge
    return float(min(max(m, 0.0), 1.0))

# Block 9 - Baseline calibration (Source flux)
BASELINE = ReactorConfig(core_radius_cm=100.0, blanket_thickness_cm=50.0, height_cm=200.0, initial_U233_frac=0.10)

for _ in range(30):
    keff_tmp, _, _, _, _ = keff_proxy(BASELINE)
    if keff_tmp < KEFF_TARGET_MAX:
        break
    BASELINE.initial_U233_frac *= 0.8  # shrink enrichment until subcritical

keff_b, _, _, _, _ = keff_proxy(BASELINE)
# desired baseline power
P_BASE_W = 3.0e8  # 300 MW_th

# compute baseline core fission macro Σf and core volume for power calibration
def core_fission_macro_and_vol(cfg: ReactorConfig) -> Tuple[float, float]:
    Vc = cylinder_vol_cm3(cfg.core_radius_cm, cfg.height_cm)
    fU = min(max(cfg.initial_U233_frac, 0.0), 1.0)
    N_u = N_HM * fU
    Sig_f_core = macro_sigma(N_u, nuc.sigma_f_U233_b)
    return Sig_f_core, Vc

Sig_f_base, Vc_base = core_fission_macro_and_vol(BASELINE)

# Solve for source flux phi_source so that:
# P = (phi_actual * Σf_core * Vc) * E_fission, and phi_actual = phi_source / (1 - keff)
phi_source = P_BASE_W * (1.0 - keff_b) / (Sig_f_base * Vc_base * E_FISSION_J)
phi_source = max(phi_source, 1e6)  # floor to avoid degenerate cases

# Block 10 - Defining a Breeding proxy (Bateman like and 24×1h stable Euler)
def breeding_proxy_1day(cfg: ReactorConfig, keff_est: float, flux_edge_frac: float) -> Tuple[float, float]:
    """
    Returns (bred_U233_atoms, bred_U233_norm_by_initial_Th).
    Uses:
      captures: Th-232 (n,γ) -> Pa-233 (assuming Th-233 decays fast)
      decay: Pa-233 -> U-233
    Rate core: R = phi_core * N_th_core * σ_ng * V_core
    Rate blanket: same with blanket flux ~ phi_core * edge_fraction
    """
    Rt = cfg.core_radius_cm + cfg.blanket_thickness_cm
    if Rt <= 0 or cfg.height_cm <= 0:
        return 0.0, 0.0

    Vc = cylinder_vol_cm3(cfg.core_radius_cm, cfg.height_cm)
    Vt = cylinder_vol_cm3(Rt, cfg.height_cm)
    Vb = max(Vt - Vc, 0.0)

    fU = min(max(cfg.initial_U233_frac, 0.0), 1.0)
    N_th_core = N_HM * (1.0 - fU)      # atoms/cm3
    N_th_blanket = N_HM               # atoms/cm3

    # actual flux in core under fixed external source
    denom = max(1e-6, (1.0 - min(keff_est, 0.999999)))
    phi_core = phi_source / denom
    phi_blanket = phi_core * max(min(flux_edge_frac, 1.0), 0.0)

    sigma_ng_cm2 = nuc.sigma_ng_Th232_b * BARN_TO_CM2

    # initial Th inventories (atoms)
    Th_core_atoms = N_th_core * Vc
    Th_blanket_atoms = N_th_blanket * Vb
    Th0 = Th_core_atoms + Th_blanket_atoms

    Pa_atoms = 0.0
    U_bred_atoms = 0.0

    dt_s = 3600.0
    steps = 24

    for _ in range(steps):
        # capture in this step (atoms/s * dt)
        cap_core = phi_core * N_th_core * sigma_ng_cm2 * Vc * dt_s
        cap_blanket = phi_blanket * N_th_blanket * sigma_ng_cm2 * Vb * dt_s

        # clamp so we never consume more Th than exists
        cap_core = min(cap_core, Th_core_atoms)
        cap_blanket = min(cap_blanket, Th_blanket_atoms)

        Th_core_atoms -= cap_core
        Th_blanket_atoms -= cap_blanket

        # Pa production (Th-233 assumed instantaneous -> Pa-233)
        Pa_atoms += (cap_core + cap_blanket)

        # Pa decay to U-233
        dec = nuc.lambda_Pa233_s * Pa_atoms * dt_s
        dec = min(dec, Pa_atoms)
        Pa_atoms -= dec
        U_bred_atoms += dec

    bred_norm = (U_bred_atoms / Th0) if Th0 > 0 else 0.0
    return float(U_bred_atoms), float(bred_norm)

# Block 11 - Dataset generation which will be used to train the Intelligent Computer System
N_SAMPLES_TARGET = 1500
NX = 36  # Recorded for reference; geometry is analytic in this surrogate

def sample_config() -> ReactorConfig:
    return ReactorConfig(
        core_radius_cm=random.uniform(50.0, 150.0),
        blanket_thickness_cm=random.uniform(0.0, 100.0),
        height_cm=random.uniform(100.0, 300.0),
        initial_U233_frac=random.uniform(0.0, 0.2),
    )

records = []
accepted = 0
attempts = 0

while accepted < N_SAMPLES_TARGET:
    attempts += 1
    cfg = sample_config()

    keff, eta, Sig_a_eff, D, B2 = keff_proxy(cfg)
    if not np.isfinite(keff):
        continue

    # hard subcritical gating
    if keff >= 1.0 or keff >= KEFF_TARGET_MAX:
        continue

    # flux shape
    Rt = cfg.core_radius_cm + cfg.blanket_thickness_cm
    x = 2.405 * (cfg.core_radius_cm / Rt) if Rt > 0 else 0.0
    edge_frac = max(float(j0(x)), 0.0)
    flux_metric = 1.0 - edge_frac
    flux_metric = float(min(max(flux_metric, 0.0), 1.0))

    # power calculation (core fissions only)
    Sig_f_core, Vc = core_fission_macro_and_vol(cfg)
    #phi_core = phi_source / max(1e-6, (1.0 - min(keff, 0.999999)))
    den = (1.0 - min(keff, KEFF_TARGET_MAX))
    den = max(den, 0.05)  # prevents extreme blow up near critical values
    phi_core = phi_source / den
    fission_rate = phi_core * Sig_f_core * Vc                      # [1/s]
    power_W = fission_rate * E_FISSION_J
    avg_pd = power_W / Vc if Vc > 0 else float("inf")

    violates_power = bool(power_W > POWER_ENV_MAX_W)
    violates_temp = bool(avg_pd > POWER_DENSITY_MAX_W_PER_CM3)
    if violates_power or violates_temp:
      continue

    # breeding (1 day)
    bred_atoms, bred_norm = breeding_proxy_1day(cfg, keff, edge_frac)

    # geometry ratios
    core_aspect = cfg.height_cm / (2.0 * cfg.core_radius_cm)
    blanket_ratio = (cfg.core_radius_cm + cfg.blanket_thickness_cm) / cfg.core_radius_cm

    # margins
    margin_keff = KEFF_TARGET_MAX - keff
    margin_power = POWER_ENV_MAX_W - power_W
    margin_temp = POWER_DENSITY_MAX_W_PER_CM3 - avg_pd

    # normalized safety margin (min of fractional margins)
    m_keff = margin_keff / KEFF_TARGET_MAX
    m_pow = margin_power / POWER_ENV_MAX_W
    m_tmp = margin_temp / POWER_DENSITY_MAX_W_PER_CM3
    safety_margin = float(min(m_keff, m_pow, m_tmp))

    rec = SampleRecord(
        core_radius_cm=cfg.core_radius_cm,
        blanket_thickness_cm=cfg.blanket_thickness_cm,
        height_cm=cfg.height_cm,
        initial_U233_frac=cfg.initial_U233_frac,

        core_aspect=float(core_aspect),
        blanket_to_core_radius_ratio=float(blanket_ratio),

        keff_est=float(keff),
        eta_est=float(eta),
        flux_shape_metric=float(flux_metric),

        breeding_proxy_atoms=float(bred_atoms),
        breeding_proxy_norm=float(bred_norm),

        total_power_W=float(power_W),
        avg_core_power_density_W_cm3=float(avg_pd),

        violates_power_envelope=violates_power,
        violates_temp_envelope=violates_temp,

        margin_keff=float(margin_keff),
        margin_power_W=float(margin_power),
        margin_temp_W_cm3=float(margin_temp),
        safety_margin=float(safety_margin),
    )
    records.append(asdict(rec))
    accepted += 1

df = pd.DataFrame.from_records(records)

# Block 11 - Save dataset
csv_path = os.path.join(OUT_DIR, "surrogate_dataset.csv")
parq_path = os.path.join(OUT_DIR, "surrogate_dataset.parquet")
npz_path  = os.path.join(OUT_DIR, "surrogate_dataset.npz")

df.to_csv(csv_path, index=False)
df.to_parquet(parq_path, index=False)
np.savez(npz_path, **{c: df[c].to_numpy() for c in df.columns})

# Block 12 - Save schema
def col_stats(col: str) -> Dict[str, Any]:
    s = df[col]
    if s.dtype == bool:
        return {"type": "bool", "mean_true": float(s.mean())}
    return {
        "type": "float" if np.issubdtype(s.dtype, np.number) else str(s.dtype),
        "min": float(np.nanmin(s.to_numpy())),
        "max": float(np.nanmax(s.to_numpy())),
        "mean": float(np.nanmean(s.to_numpy())),
    }

schema = {
    "meta": {
        "name": "Chapter 5 Gedanken V2.0 — PHWR-style subcritical surrogate dataset (U-233/Th)",
        "seed": SEED,
        "n_samples": int(len(df)),
        "attempts": int(attempts),
        "nx_reference": NX,
        "subcritical_gate": {"keff_lt": 1.0, "keff_target_max": KEFF_TARGET_MAX},
        "envelopes": {"power_W_max": POWER_ENV_MAX_W, "power_density_W_cm3_max": POWER_DENSITY_MAX_W_PER_CM3},
    },
    "columns": {
        "core_radius_cm": {"unit": "cm", "desc": "Core fuel region radius", **col_stats("core_radius_cm")},
        "blanket_thickness_cm": {"unit": "cm", "desc": "Radial blanket thickness", **col_stats("blanket_thickness_cm")},
        "height_cm": {"unit": "cm", "desc": "Active height of core/blanket (same height)", **col_stats("height_cm")},
        "initial_U233_frac": {"unit": "fraction", "desc": "Initial heavy-metal atomic fraction of U-233 in core fuel", **col_stats("initial_U233_frac")},
        "core_aspect": {"unit": "-", "desc": "H / (2*R_core)", **col_stats("core_aspect")},
        "blanket_to_core_radius_ratio": {"unit": "-", "desc": "(R_core + t_blanket) / R_core", **col_stats("blanket_to_core_radius_ratio")},
        "keff_est": {"unit": "-", "desc": "One-group diffusion proxy k_eff", **col_stats("keff_est")},
        "eta_est": {"unit": "-", "desc": "Infinite-medium proxy eta = nuΣf/Σa (volume-weighted)", **col_stats("eta_est")},
        "flux_shape_metric": {"unit": "-", "desc": "1 - J0(2.405*R_core/R_total), clamped [0,1]", **col_stats("flux_shape_metric")},
        "breeding_proxy_atoms": {"unit": "atoms", "desc": "Bred U-233 atoms after 1 day (Th->Pa->U), proxy", **col_stats("breeding_proxy_atoms")},
        "breeding_proxy_norm": {"unit": "-", "desc": "Bred U-233 normalized by initial Th inventory", **col_stats("breeding_proxy_norm")},
        "total_power_W": {"unit": "W", "desc": "Thermal power proxy from fission rate in core", **col_stats("total_power_W")},
        "avg_core_power_density_W_cm3": {"unit": "W/cm^3", "desc": "Power density proxy (thermal constraint)", **col_stats("avg_core_power_density_W_cm3")},
        "violates_power_envelope": {"unit": "bool", "desc": "True if total_power exceeds envelope limit", **col_stats("violates_power_envelope")},
        "violates_temp_envelope": {"unit": "bool", "desc": "True if avg core power density exceeds envelope limit", **col_stats("violates_temp_envelope")},
        "margin_keff": {"unit": "-", "desc": "KEFF_TARGET_MAX - keff_est", **col_stats("margin_keff")},
        "margin_power_W": {"unit": "W", "desc": "POWER_ENV_MAX_W - total_power", **col_stats("margin_power_W")},
        "margin_temp_W_cm3": {"unit": "W/cm^3", "desc": "POWER_DENSITY_MAX - avg_power_density", **col_stats("margin_temp_W_cm3")},
        "safety_margin": {"unit": "fraction", "desc": "min of normalized margins (keff, power, temp)", **col_stats("safety_margin")},
    },
}

with open(os.path.join(OUT_DIR, "dataset_schema.json"), "w", encoding="utf-8") as f:
    json.dump(schema, f, indent=2)

# Final Block - Printing summary
print("Chapter 5 Gedanken V2.0 Output (ENDF) Run 1:")
print(f"Saved to: {OUT_DIR}/")
print(f"Samples: {len(df)} | Attempts: {attempts} | Acceptance: {len(df)/attempts:.3f}")
print("Nuclear data used:")
print(json.dumps(nuc.source_info, indent=2))
print("Key stats:")
print(df[["keff_est","eta_est","breeding_proxy_norm","flux_shape_metric","total_power_W","avg_core_power_density_W_cm3"]].describe().loc[["min","mean","max"]])


Chapter 5 Gedanken V2.0 Output (ENDF) Run 1:
Saved to: step5_dataset_out/
Samples: 1500 | Attempts: 14932 | Acceptance: 0.100
Nuclear data used:
{
  "U233_fission": {
    "source": "ENDF",
    "library": "ENDF/B-VIII.0",
    "E_eV": 0.0253,
    "note": "manual thermal point 2.53e-8 MeV"
  },
  "U233_capture": {
    "source": "ENDF",
    "library": "ENDF/B-VIII.0",
    "E_eV": 0.0253,
    "note": "manual thermal point 2.53e-8 MeV"
  },
  "Th232_capture": {
    "source": "ENDF",
    "library": "ENDF/B-VIII.0",
    "E_eV": 0.0253,
    "note": "manual thermal point 2.53e-8 MeV"
  }
}
Key stats:
      keff_est   eta_est  breeding_proxy_norm  flux_shape_metric  \
min   0.000834  0.000835         2.892047e-09           0.159346   
mean  0.506450  0.506928         9.084824e-09           0.483611   
max   0.976307  0.978154         1.555792e-07           0.998144   

      total_power_W  avg_core_power_density_W_cm3  
min    5.758628e+04                      0.005613  
mean   8.584999e+07      